# नमुना ०१: OpenAI SDK च्या माध्यमातून जलद संवाद

ही नोटबुक Microsoft Foundry Local सह OpenAI SDK वापरून जलद संवाद कसा साधायचा हे दाखवते.

## आढावा

या उदाहरणात दाखवले आहे:
- Foundry Local सह OpenAI Python SDK वापरणे
- Azure OpenAI आणि स्थानिक Foundry कॉन्फिगरेशन हाताळणे
- योग्य त्रुटी हाताळणी आणि पर्यायी रणनीती अंमलात आणणे
- सेवा व्यवस्थापनासाठी FoundryLocalManager वापरणे


## पूर्वअट

याची खात्री करा की तुमच्याकडे:
1. Foundry Local स्थापित आणि चालू आहे
2. एक मॉडेल लोड केले आहे (उदा., `foundry model run phi-4-mini`)
3. आवश्यक Python पॅकेजेस स्थापित केले आहेत

### Dependencies स्थापित करा


In [ ]:
# Install required packages if not already installed
!pip install openai foundry-local-sdk

## आवश्यक ग्रंथालये आयात करा


In [ ]:
import os
import sys
from openai import OpenAI

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, will use manual configuration")

## कॉन्फिगरेशन

तुमची कॉन्फिगरेशन सेट करा. तुम्ही खालीलपैकी कोणताही पर्याय वापरू शकता:
1. **Azure OpenAI** (क्लाउड-आधारित)
2. **Foundry Local** (स्थानिक इनफरन्स)

### पर्याय 1: Azure OpenAI कॉन्फिगरेशन


In [ ]:
# Azure OpenAI Configuration (uncomment and set your values)
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com"
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["MODEL"] = "your-deployment-name"

### पर्याय 2: फाउंड्री स्थानिक कॉन्फिगरेशन


In [ ]:
# Foundry Local Configuration
MODEL_ALIAS = "phi-4-mini"  # Change this to your preferred model
BASE_URL = "http://localhost:8000"  # Default Foundry Local URL
API_KEY = ""  # Usually not needed for local

## क्लायंट प्रारंभ

तुमच्या कॉन्फिगरेशनवर आधारित OpenAI क्लायंट तयार करा:


In [ ]:
def create_client():
    """Create and configure OpenAI client."""
    
    # Check for Azure OpenAI configuration
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
    
    if azure_endpoint and azure_api_key:
        # Azure OpenAI path
        model = os.environ.get("MODEL", "your-deployment-name")
        client = OpenAI(
            base_url=f"{azure_endpoint}/openai",
            api_key=azure_api_key,
            default_query={"api-version": azure_api_version},
        )
        print(f"🌐 Using Azure OpenAI with model: {model}")
        return client, model
    
    # Foundry Local path
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            manager = FoundryLocalManager(MODEL_ALIAS)
            model_info = manager.get_model_info(MODEL_ALIAS)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key  # API key is not required for local usage
            )
            model = model_info.id
            print(f"🏠 Using Foundry Local SDK with model: {model}")
            return client, model
        except Exception as e:
            print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
    
    # Fallback to manual configuration
    client = OpenAI(
        base_url=f"{BASE_URL}/v1",
        api_key=API_KEY
    )
    model = MODEL_ALIAS
    print(f"🔧 Using manual configuration with model: {model}")
    return client, model

# Initialize the client
client, model = create_client()

## साधा चॅट उदाहरण

चला एक सोपी चॅट संवादाची चाचणी करूया:


In [ ]:
def chat_with_model(prompt, max_tokens=128):
    """Send a chat message to the model and get a response."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test with a simple prompt
prompt = "Say hello from Foundry Local and explain what you are in one sentence."
print(f"👤 User: {prompt}")
print("\n🤖 Assistant:")
response = chat_with_model(prompt)
print(response)

## संवादात्मक चॅट सत्र

मॉडेल कसे प्रतिसाद देते हे पाहण्यासाठी विविध प्रकारचे प्रॉम्प्ट वापरून पहा:


In [ ]:
# Example prompts to try
example_prompts = [
    "What are the benefits of edge AI?",
    "Explain the difference between local and cloud AI inference.",
    "Write a short Python function to calculate the factorial of a number.",
    "What is Microsoft Foundry Local?"
]

for i, prompt in enumerate(example_prompts, 1):
    print(f"\n{'='*50}")
    print(f"Example {i}: {prompt}")
    print(f"{'='*50}")
    
    response = chat_with_model(prompt, max_tokens=200)
    print(response)

## प्रगत वापर: स्ट्रीमिंग प्रतिसाद

लांब प्रतिसादांसाठी, स्ट्रीमिंग वापरकर्त्याचा अनुभव अधिक चांगला देऊ शकतो:


In [ ]:
def chat_with_streaming(prompt, max_tokens=300):
    """Send a chat message with streaming response."""
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            stream=True
        )
        
        print("🤖 Assistant (streaming):")
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        print("\n")  # New line after streaming
        return full_response
    except Exception as e:
        return f"Error: {e}"

# Test streaming with a longer prompt
streaming_prompt = "Explain the advantages of running AI models locally on device versus in the cloud. Include privacy, latency, and cost considerations."
print(f"👤 User: {streaming_prompt}\n")
streaming_response = chat_with_streaming(streaming_prompt)

## त्रुटी हाताळणी आणि निदान

सेवा स्थिती आणि उपलब्ध मॉडेल्स तपासूया:


In [ ]:
def check_service_health():
    """Check Foundry Local service health."""
    try:
        # Try to list available models
        models_response = client.models.list()
        models = [model.id for model in models_response.data]
        
        print("✅ Service is healthy")
        print(f"📋 Available models: {models}")
        print(f"🎯 Current model: {model}")
        
        if model in models:
            print("✅ Current model is available")
        else:
            print("⚠️ Current model may not be loaded")
            
    except Exception as e:
        print(f"❌ Service check failed: {e}")
        print("\n🔧 Troubleshooting tips:")
        print("1. Make sure Foundry Local is running")
        print("2. Check if a model is loaded: foundry service ps")
        print("3. Verify the endpoint URL is correct")

check_service_health()

## कस्टम प्रॉम्प्ट चाचणी

खालील सेल वापरून तुमचे स्वतःचे प्रॉम्प्ट तपासा:


In [ ]:
# Enter your custom prompt here
custom_prompt = "Write a haiku about artificial intelligence running on edge devices."

print(f"👤 User: {custom_prompt}\n")
custom_response = chat_with_model(custom_prompt, max_tokens=100)
print(f"🤖 Assistant: {custom_response}")

## सारांश

या नोटबुकमध्ये खालील गोष्टी दाखवल्या आहेत:

1. **✅ क्लायंट सेटअप**: Foundry Local सह OpenAI SDK कसे कॉन्फिगर करायचे
2. **✅ बेसिक चॅट**: सोप्या विनंती-प्रतिसाद संवाद
3. **✅ स्ट्रीमिंग**: रिअल-टाइम प्रतिसाद निर्मिती
4. **✅ एरर हँडलिंग**: मजबूत त्रुटी हाताळणी आणि निदान
5. **✅ सेवा आरोग्य**: मॉडेलची उपलब्धता आणि सेवेची स्थिती तपासणे

### पुढील पावले

- **सॅम्पल 02**: Azure OpenAI समर्थनासह प्रगत SDK एकत्रीकरण
- **सॅम्पल 04**: Chainlit चॅट अॅप्लिकेशन तयार करणे
- **सॅम्पल 05**: मल्टी-एजंट ऑर्केस्ट्रेशन प्रणाली
- **सॅम्पल 06**: बुद्धिमान मॉडेल रूटिंग

### Foundry Local चे मुख्य फायदे

- 🔒 **गोपनीयता**: डेटा तुमच्या डिव्हाइसच्या बाहेर जात नाही
- ⚡ **गती**: कमी विलंब स्थानिक अनुमान
- 💰 **खर्च**: API वापराचा कोणताही खर्च नाही
- 🔌 **ऑफलाइन**: इंटरनेट कनेक्शनशिवाय कार्य करते
- 🛠️ **सुसंगतता**: OpenAI-सुसंगत API
